In [32]:
import cv2 
import numpy as np 
import matplotlib.pyplot as plt 

In [33]:
SIZE_IMAGE = 20
NUM_CLASSES = 10

def load_digits_and_labels(big_image):
    """Returns all the digits and create respective labels"""

    digits_img = cv2.imread(big_image, 0)

    # Get all the digit images from the 'big' image:
    number_rows = digits_img.shape[1] / SIZE_IMAGE
    rows = np.vsplit(digits_img, digits_img.shape[0] / SIZE_IMAGE)

    digits = []
    for row in rows:
        row_cells = np.hsplit(row, number_rows)
        for digit in row_cells:
            digits.append(digit)
    digits = np.array(digits)

    # Create the labels for each image:
    labels = np.repeat(np.arange(NUM_CLASSES), len(digits) / NUM_CLASSES)
    return digits, labels

In [34]:
def deskew(img):
    """Pre-processing of the images"""

    m = cv2.moments(img)
    if abs(m['mu02']) < 1e-2:
        return img.copy()
    skew = m['mu11'] / m['mu02']
    M = np.float32([[1, skew, -0.5 * SIZE_IMAGE * skew],[0, 1,0]])
    img = cv2.warpAffine(img, M, (SIZE_IMAGE, SIZE_IMAGE), flags=cv2.WARP_INVERSE_MAP | cv2.INTER_LINEAR)
    return img

In [35]:
def svm_init(C=12.5, gamma=0.50625):
    """Creates empty model and assigns main parameters"""

    model = cv2.ml.SVM_create()
    model.setGamma(gamma)
    model.setC(C)
    model.setKernel(cv2.ml.SVM_RBF)
    model.setType(cv2.ml.SVM_C_SVC)
    model.setTermCriteria((cv2.TERM_CRITERIA_MAX_ITER, 100, 1e-6))

    return model

In [36]:
def svm_train(model, samples, responses):
    """Train the model using the samples and the responses"""

    model.train(samples, cv2.ml.ROW_SAMPLE, responses)
    return model

In [37]:
def svm_predict(model, samples):
    """Predicts the response based on the trained model"""

    return model.predict(samples)[1].ravel()

In [38]:
def svm_evaluate(model, samples, labels):
    """Calculates the accuracy of the trained model"""

    predictions = svm_predict(model, samples)
    accuracy = (labels == predictions).mean()
    print("Percentage Accuracy: %.2f %%" %(accuracy * 100))

In [39]:
def get_hog():
    """Get Hog Descriptor"""
    hog = cv2.HOGDescriptor((SIZE_IMAGE, SIZE_IMAGE), (8, 8), (4, 4), (8, 8), 9, 1, -1, 0, 0.2, 1, 64, True)

    print("Get Descriptor Size: {}".format(hog.getDescriptorSize()))
    return hog

In [40]:
def raw_pixels(img):
    return img.flatten()

In [41]:
# Load all the digits and the corresponding labels:
digits, labels = load_digits_and_labels('images/digits.png')

# Shuffle data
# Constructs a random number generator:
rand = np.random.RandomState(1234)
# Randomly permute the sequence:
shuffle = rand.permutation(len(digits))
digits, labels = digits[shuffle], labels[shuffle]

# HoG feature descriptor:
hog = get_hog()

Get Descriptor Size: 144


In [42]:
#Computing the DESCRIPTORS for all the image

hog_descriptors = []

for img in digits:
    hog_descriptors.append(hog.compute(deskew(img)))
hog_descriptors = np.squeeze(hog_descriptors)

#Splitting data 
partition = int(0.5 * len(hog_descriptors))
hog_descriptors_train, hog_descriptors_test = np.split(hog_descriptors, [partition])
labels_train, labels_test = np.split(labels ,[partition])

In [43]:
print('Training SVM model ...')
model = svm_init(C=12.5, gamma=0.50625)
svm_train(model, hog_descriptors_train, labels_train)

print('Evaluating model ... ')
svm_evaluate(model, hog_descriptors_test, labels_test)

Training SVM model ...
Evaluating model ... 
Percentage Accuracy: 98.60 %
